In [25]:
from datetime import datetime

import polars as pl
from polygon import RESTClient

from src.utils import load_polygon_api_key

In [24]:
client = RESTClient(api_key=load_polygon_api_key())

In [28]:
def get_aggregates(ticker, from_, to, multiplier=5, timespan="minute"):
    aggs = []
    for a in client.list_aggs(
        ticker=ticker,
        multiplier=multiplier,
        timespan=timespan,
        from_=from_,
        to=to,
        limit=50000,
    ):
        aggs.append(a)
    return aggs

In [29]:
# aggs = []
# for a in client.list_aggs(
#     ticker="SPY",
#     multiplier=1,
#     timespan="hour",
#     from_="2022-01-01",
#     to="2022-01-31",
#     limit=50000,
# ):
#     aggs.append(a)
    
# print(aggs)
aggs = get_aggregates("SPY", "2022-01-01", "2022-01-31", multiplier=1, timespan="hour")

In [20]:
data = aggs[0]
print(data)
timestamp = data.timestamp
print(timestamp)

datetime.fromtimestamp(timestamp / 1000)


Agg(open=476.38, high=477.25, low=476.38, close=477.25, volume=57805, vwap=476.8048, timestamp=1641200400000, transactions=556, otc=None)
1641200400000


datetime.datetime(2022, 1, 3, 18, 0)

In [21]:
def transfrom_api_to_dict(data):
    # exclude otc
    return {
        "open": data.open,
        "high": data.high,
        "low": data.low,
        "close": data.close,
        "volume": data.volume,
        "vwap": data.vwap,
        "timestamp": datetime.fromtimestamp(data.timestamp / 1000),
        "transactions": data.transactions,
    }

In [22]:
transfrom_api_to_dict(aggs[0])

{'open': 476.38,
 'high': 477.25,
 'low': 476.38,
 'close': 477.25,
 'volume': 57805,
 'vwap': 476.8048,
 'timestamp': datetime.datetime(2022, 1, 3, 18, 0),
 'transactions': 556}

In [23]:
aggs_list = [transfrom_api_to_dict(data) for data in aggs]
aggs_list

[{'open': 476.38,
  'high': 477.25,
  'low': 476.38,
  'close': 477.25,
  'volume': 57805,
  'vwap': 476.8048,
  'timestamp': datetime.datetime(2022, 1, 3, 18, 0),
  'transactions': 556},
 {'open': 477.24,
  'high': 477.76,
  'low': 477.15,
  'close': 477.76,
  'volume': 36501,
  'vwap': 477.5092,
  'timestamp': datetime.datetime(2022, 1, 3, 19, 0),
  'transactions': 395},
 {'open': 477.76,
  'high': 478.09,
  'low': 477.76,
  'close': 478,
  'volume': 74462,
  'vwap': 477.9857,
  'timestamp': datetime.datetime(2022, 1, 3, 20, 0),
  'transactions': 492},
 {'open': 478.02,
  'high': 478.06,
  'low': 477.33,
  'close': 477.41,
  'volume': 156478,
  'vwap': 477.6712,
  'timestamp': datetime.datetime(2022, 1, 3, 21, 0),
  'transactions': 1402},
 {'open': 477.9504,
  'high': 477.99,
  'low': 476.53,
  'close': 476.56,
  'volume': 517794,
  'vwap': 477.2607,
  'timestamp': datetime.datetime(2022, 1, 3, 22, 0),
  'transactions': 2844},
 {'open': 476.53,
  'high': 477.72,
  'low': 473.85,
  'c

In [26]:
df = pl.from_dicts(aggs_list)
df

open,high,low,close,volume,vwap,timestamp,transactions
f64,f64,f64,f64,f64,f64,datetime[μs],i64
476.38,477.25,476.38,477.25,57805.0,476.8048,2022-01-03 18:00:00,556
477.24,477.76,477.15,477.76,36501.0,477.5092,2022-01-03 19:00:00,395
477.76,478.09,477.76,478.0,74462.0,477.9857,2022-01-03 20:00:00,492
478.02,478.06,477.33,477.41,156478.0,477.6712,2022-01-03 21:00:00,1402
477.9504,477.99,476.53,476.56,517794.0,477.2607,2022-01-03 22:00:00,2844
476.53,477.72,473.85,474.9266,1.0602782e7,475.8451,2022-01-03 23:00:00,107079
474.94,476.78,474.77,476.17,9.474299e6,475.8476,2022-01-04 00:00:00,94361
476.16,476.25,474.45,475.36,6.846941e6,475.2563,2022-01-04 01:00:00,68695
475.32,475.995,475.03,475.6,3.690321e6,475.6205,2022-01-04 02:00:00,40691


In [ ]:
aggs_feb = get_aggregates("SPY", "2022-02-01", "2022-02-28", multiplier=1, timespan="hour")
df_feb = 